# 机器人强化学习之使用 OpenAI Gym 教程与笔记

>除了试图直接去建立一个可以模拟成人大脑的程序之外， 为什么不试图建立一个可以模拟小孩大脑的程序呢?如果它接 受适当的教育，就会获得成人的大脑。 — 阿兰·图灵

## 介绍

强化学习 (Reinforcement learning) 是机器学习的一个子领域用于制定决策和运动自由度控制。强化学习主要研究在复杂未知的环境中，智体(agent)实现某个目标。强化学习最引人入胜的两个特点是

- **强化学习非常通用，可以用来解决需要作出一些列决策的所有问题：**例如，训练机器人跑步和弹跳，制定商品价格和库存管理，玩 Atari 游戏和棋盘游戏等等。

- **强化学习已经可以在许多复杂的环境中取得较好的实验结果：**例如 Deep RL 的 Alpha Go等

[Gym](https://gym.openai.com/docs/) 是一个研究和开发强化学习相关算法的仿真平台。

- 无需智体先验知识；
- 兼容常见的数值运算库如 TensorFlow、Theano 等

## Gym 的一个最小例子 `CartPole-v0`



### 车杆模型

> 出自 Barto, Sutton, and Anderson 的 `cart-pole problem`

In [5]:
import gym
%matplotlib inline
env = gym.make('CartPole-v0')
observation = env.reset()
for _ in range(1000):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action) # take a random action
    if done: 
        env.render()
        break

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


**运行效果**

![](https://ws1.sinaimg.cn/large/c3a916a7gy1ftmzl7ss9aj20a804a0sj.jpg)

至此，第一个 Hello world 就算正式地跑起来了！

## 观测(Observations)

在第一个小栗子中，使用了 `env.step()` 函数来对每一步进行仿真，在 Gym 中，`env.step()` 会返回 4 个参数：

- **观测** Observation (Object)：当前 step 执行后，环境的观测(类型为对象)。例如，从相机获取的像素点，机器人各个关节的角度或棋盘游戏当前的状态等；

- **奖励** Reward (Float): 执行上一步动作(action)后，智体(agent)获得的奖励(浮点类型)，不同的环境中奖励值变化范围也不相同，但是强化学习的目标就是使得总奖励值最大；

- **完成** Done (Boolen): 表示是否需要将环境重置 `env.reset`。大多数情况下，当 `Done` 为 `True` 时，就表明当前回合(episode)或者试验(tial)结束。例如当机器人摔倒或者掉出台面，就应当终止当前回合进行重置(reset);

- **信息** Info (Dict): 针对调试过程的诊断信息。在标准的智体仿真评估当中不会使用到这个 info，具体用到的时候再说。


总结来说，这就是一个强化学习的基本流程，在每个时间点上，智体执行 action，环境返回上一次 action 的观测和奖励，用图表示为

![智体与环境交互](https://ws1.sinaimg.cn/large/c3a916a7gy1ftn0jm54q0j20gm082t8y.jpg)

在 Gym 仿真中，每一次回合开始，需要先执行 `reset()` 函数，返回初始观测信息，然后根据标志位 `done` 的状态，来决定是否进行下一次回合。代码表示为

In [ ]:
import gym
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

仿真截图如下

![](https://ws1.sinaimg.cn/large/c3a916a7gy1fto3j0pf0yj20go0bqdfu.jpg)

每次 `action` 前，将上一次 `observation` 打印，可以得到打印日志如下

```
[ 0.0349103   1.14771978 -0.03934506 -1.64631971]
[ 0.0578647   1.34327926 -0.07227145 -1.95099638]
[ 0.08473028  1.14899616 -0.11129138 -1.68156178]
[ 0.1077102   0.95532555 -0.14492261 -1.42550525]
[ 0.12681672  1.15191062 -0.17343272 -1.75974995]
[ 0.14985493  0.95912509 -0.20862772 -1.52564382]
Episode finished after 16 timesteps
[ 0.03628829 -0.03189712 -0.01997778  0.02529094]
[ 0.03565035 -0.22672696 -0.01947196  0.31160431]
[ 0.03111581 -0.42156616 -0.01323988  0.59808332]
[ 0.02268449 -0.61650037 -0.00127821  0.8865666 ]
```


## 空间（Spaces）

在前面的两个小栗子中，每次执行的动作(action)都是从环境动作空间中随机进行选取的，但是这些动作 (action) 是什么?在 Gym 的仿真环境中，有运动空间 `action_space` 和观测空间 `observation_space` 两个指标，程序中被定义为 `Space` 类型，用于描述有效的运动和观测的格式和范围。下面是一个代码示例

In [3]:
import gym
env = gym.make('CartPole-v0')
print(env.action_space)
#> Discrete(2)
print(env.observation_space)
#> Box(4,)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Discrete(2)
Box(4,)


从程序运行结果来看

- `action_space` 是一个离散 `Discrete` 类型，从 [discrete.py](https://github.com/openai/gym/blob/master/gym/spaces/discrete.py) 源码可知，范围是一个 `{0,1,...,n-1}` 长度为 `n` 的非负整数集合，在 `CartPole-v0` 例子中，动作空间表示为 `{0,1}`。

- `observation_space` 是一个 `Box` 类型，从 [box.py](https://github.com/openai/gym/blob/master/gym/spaces/box.py) 源码可知，表示一个 `n` 维的盒子，所以在上一节打印出来的 `observation` 是一个长度为 4 的数组。数组中的每个元素都具有上下界。

In [2]:
print(env.observation_space.high)
print(env.observation_space.low)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


利用运动空间和观测空间的定义和范围，可以将代码写得更加通用。在许多仿真环境中，`Box` 和 `Discrete` 是最常见的空间描述，在智体每次执行动作时，都属于这些空间范围内，代码示例为

In [2]:
from gym import spaces
space = spaces.Discrete(8) # Set with 8 elements {0, 1, 2, ..., 7}
x = space.sample()
print(space.contains(x)) 
print(space.n == 8) 

True
True


在 `CartPole-v0` 栗子中，运动只能选择左和右，分别用 `{0,1}` 表示

## Gym 中可用的环境

Gym 中从简单到复杂，包含了许多经典的[仿真环境](https://gym.openai.com/envs/#classic_control)和各种数据，其中包括

- 经典控制和文字游戏：经典的强化学习示例，方便入门；

- 算法：从例子中学习强化学习的相关算法，在 Gym 的仿真算法中，由易到难方便新手入坑；

- 雅达利游戏：利用强化学习来玩雅达利的游戏。Gym 中集成了对强化学习有着重要影响的 [Arcade Learning Environment](http://www.arcadelearningenvironment.org/)，并且方便用户安装；

- 2D 和 3D 的机器人：这个是我一直很感兴趣的一部分，在 Gym 中控制机器人进行仿真。需要利用第三方的物理引擎如 `MuJoCo` 。

![](https://ws1.sinaimg.cn/large/c3a916a7gy1fto4z7mno0j20oa0ikqbs.jpg)

## 注册表

Gym 是一个包含各种各样强化学习仿真环境的大集合，并且封装成通用的接口暴露给用户，查看所有环境的代码如下：

In [ ]:
from gym import envs
print(envs.registry.all())

```
dict_values([EnvSpec(Copy-v0), EnvSpec(RepeatCopy-v0), EnvSpec(ReversedAddition-v0), EnvSpec(ReversedAddition3-v0), EnvSpec(DuplicatedInput-v0), EnvSpec(Reverse-v0), EnvSpec(CartPole-v0), EnvSpec(CartPole-v1), EnvSpec(MountainCar-v0), EnvSpec(MountainCarContinuous-v0), EnvSpec(Pendulum-v0), EnvSpec(Acrobot-v1), EnvSpec(LunarLander-v2), EnvSpec(LunarLanderContinuous-v2), EnvSpec(BipedalWalker-v2),...
```

Gym 支持将用户制作的环境写入到注册表中，需要执行 `gym.make()` 和在启动时注册 `register`，例如

```python
register(
    id='CartPole-v0',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    max_episode_steps=200,
    reward_threshold=195.0,
)
```


## 参考链接

- [https://gym.openai.com/docs/](https://gym.openai.com/docs/)

- [https://nndl.github.io/](https://nndl.github.io/) 

## 结语

emmmm ... 第一篇强化学习入坑笔记写完，大多是从官方文档看过来的加上了一点点自己的理解，建议文档这东西还是直接看官方的吧，原汁原味

## 关于作者

- 神奇的战士
- 博客：[http://thinkhard.tech/](http://thinkhard.tech/)
- Github: [https://github.com/wangshub](https://github.com/wangshub)
- 微信公众号：**舒林笑笑生**

